# Getting `bboxes` from rs19 for semi-automatic marking

In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import json

import os
from pathlib import Path
import shutil

from pprint import pprint
from tqdm import tqdm
import locale

def getpreferredencoding(do_setlocale = True):
    """
    Для корректного чтения файлов .npy
    """
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding
print(locale.getpreferredencoding())

In [ ]:
with open(r"C:\Users\ivano\Desktop\Синара\Синара\railsem_true\jsons\rs19_val\rs00009.json") as file:
    data = file.read()

dic = json.loads(data)

## Function for iterating through all the pictures and annotations in the folders `/images/` and `/annotations/`
## for reading `bboxes` and cutting them from the original image to create a dataset for semi-automatic. polygon markings

#### 4 classes of switches:

 - ~~switch-indicator~~
 - 'switch-left'
 - 'switch-right'
 - ~~'switch-static'~~
 - 'switch-unknown'

In [ ]:
#cut bboxes from the original images of the rs19 dataset and add 4% of the width and height to the edges

path_to_imgs = Path('../../data/jpgs/rs19_val')
path_to_jsons = Path('../../data/jsons/rs19_val')
classnames = ['switch-left', 'switch-right', 'switch-unknow']
imgs_list = os.listdir(path_to_imgs)    
jsons_list = os.listdir(path_to_jsons)

increase_bboxes = 2

for filename in tqdm(jsons_list, 'files processed'):
    with open(Path(path_to_jsons, filename)) as file:
        data = json.loads(file.read())

    img_path = str(Path(path_to_imgs, filename[:-4] + 'jpg'))
    img = cv2.imread(img_path)

    wh_inc = 1 + increase_bboxes / 100
    xy_inc = 1 - increase_bboxes / 100

    for idx, obj in enumerate(data['objects']):
        if obj['label'] in classnames:
            try:
                x, y, w, h = obj['boundingbox']

                if increase_bboxes != 0:
                    x, y = int(xy_inc * x), int(xy_inc * y)
                    w, h = int(wh_inc * w), int(wh_inc * h)
                
                roi_color = img[y:h, x:w]
                cv2.imwrite(f'../../data/bboxes_switches/{data["frame"]}_{idx}.jpg', roi_color)
            except: 
                continue

In [ ]:
len(os.listdir('../../data/bboxes_switches'))

In [ ]:
# shutil.rmtree('../../data/bboxes_switches/')
# os.makedirs('../../data/bboxes_switches/')

In [ ]:
#Cut out bboxes where there are no switches to add a background class

path_to_imgs = Path('../../data/jpgs/rs19_val')
path_to_jsons = Path('../../data/jsons/rs19_val')
stop_word = 'switch'    
imgs_list = os.listdir(path_to_imgs)    
jsons_list = os.listdir(path_to_jsons)

increase_bboxes = 2

for filename in tqdm(jsons_list, 'files processed'):
    with open(Path(path_to_jsons, filename)) as file:
        data = json.loads(file.read())

    img_path = str(Path(path_to_imgs, filename[:-4] + 'jpg'))
    img = cv2.imread(img_path)

    wh_inc = 1 + increase_bboxes / 100
    xy_inc = 1 - increase_bboxes / 100

    for idx, obj in enumerate(data['objects']):
        if stop_word not in obj['label']:
            try:
                x, y, w, h = obj['boundingbox']

                if increase_bboxes != 0:
                    x, y = int(xy_inc * x), int(xy_inc * y)
                    w, h = int(wh_inc * w), int(wh_inc * h)

                lab = obj['label']
                roi_color = img[y:h, x:w]
                cv2.imwrite(f'../../data/bboxes_other/{data["frame"]}_{lab}_{idx}.jpg', roi_color)
            except: 
                continue

In [ ]:
len(os.listdir('../../data/bboxes_other'))